In [ ]:
import pyspark
sc = pyspark.SparkContext('local[*]')

In [ ]:
!rm ./metastore_db/*.lck
examples_folder = "/usr/local/spark/examples/src/main/resources/"

In [ ]:
from pyspark.sql import SQLContext
sqlc = SQLContext(sc)

## Parquet Files

In [ ]:
df = sqlc.read.load(examples_folder + "users.parquet")
df.show()

In [ ]:
!rm -rf namesAndFavColors.parquet/
df.select("name", "favorite_color").write.save("namesAndFavColors.parquet")

In [ ]:
!ls -l namesAndFavColors.parquet/

In [ ]:
sqlc.read.format("org.apache.spark.sql.parquet").load(examples_folder + "users.parquet")

In [ ]:
sqlc.read.format("parquet").load(examples_folder + "users.parquet")

In [ ]:
sqlc.read.parquet(examples_folder + "users.parquet")

## JSON Files

In [ ]:
df_json = sqlc.read.format("json").load(examples_folder + "people.json")
df_json.show()

In [ ]:
df_json.printSchema()

In [ ]:
json_strings = df_json.toJSON()
json_strings.collect()

In [ ]:
!rm -rf namesAndAges.parquet/
df_json.select("name", "age").write.format("parquet").save("namesAndAges.parquet")

In [ ]:
!ls -l namesAndAges.parquet

In [ ]:
!rm -rf namesAndAgesCoalesced.parquet/
df_json.select("name", "age").coalesce(1).write.format("parquet").save("namesAndAgesCoalesced.parquet")

In [ ]:
!ls -l namesAndAgesCoalesced.parquet

In [ ]:
df_json.write.mode("error").parquet("namesAndAges.parquet")
# This IS supposed to throw an exception, as it is trying to write an already existent file!

In [ ]:
df_json.write.mode("overwrite").parquet("namesAndAges.parquet")

## SQL on Files

In [ ]:
df = sqlc.sql("SELECT * FROM json.`/usr/local/spark/examples/src/main/resources/people.json` WHERE age = 19")
df.show()

## MySQL

In [ ]:
df_mysql = sqlc.read.format("jdbc") \
                .option("url", "jdbc:mysql://localhost:3306/test") \
                .option("driver", "com.mysql.jdbc.Driver") \
                .option("dbtable", "users") \
                .option("user", "root") \
                .option("password", "").load()

In [ ]:
df_mysql.show()

In [ ]:
df_mysql.printSchema()

In [ ]:
df_mysql.registerTempTable("users")

In [ ]:
sqlc.sql("select * from users").collect()

In [ ]:
sc.stop()